In [1]:
import torch
from torch import nn,Tensor
from d2l import torch as d2l

In [2]:
def corr2d(X:Tensor,K:Tensor):
  h,w = K.shape
  Y = torch.zeros((X.shape[0] - h +1,X.shape[1]-w+1))
  for i in range(Y.shape[0]):
    for j in range(Y.shape[1]):
      Y[i,j] = (X[i:i+h,j:j+w] * K).sum()
  return Y
      

In [3]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X,K=K)

tensor([[19., 25.],
        [37., 43.]])

In [4]:
class Conv2D(nn.Module):
  def __init__(self, kernel_size) -> None:
    super().__init__()
    self.weight = nn.Parameter(torch.rand(kernel_size))
    self.bias = nn.Parameter(torch.zeros(1))
    
  def forward(self,X:Tensor):
    return corr2d(X,self.weight) + self.bias

In [5]:
X = torch.ones(6,8)
X[:,2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [9]:
# 互相关运算时，相邻两元素相同，输出零，否则输出非零
K = torch.Tensor([[1.0,-1.0]])

In [11]:
# 如果颜色有变化就是边缘，所以输出的1为白色到黑色的边缘，输出的零为黑色到白色的边缘
Y = corr2d(X,K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [12]:
# 只可检测垂直边缘，不可检测水平边缘
corr2d(X.t(),K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [19]:
conv2d = nn.Conv2d(1,1,kernel_size=(1,2),bias=False)
X = X.reshape((1,1,6,8))
Y = Y.reshape((1,1,6,7))
lr = 3e-2
opt = torch.optim.SGD(conv2d.parameters(),lr=lr)
for i in range(10):
  opt.zero_grad()
  Y_hat  = conv2d(X)
  l = (Y_hat-Y) ** 2
  l.sum().backward()
  opt.step()
  if (i + 1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')

epoch 2, loss 10.312
epoch 4, loss 3.214
epoch 6, loss 1.147
epoch 8, loss 0.441
epoch 10, loss 0.176


In [20]:
conv2d.weight.data.reshape((1,2))

tensor([[ 1.0323, -0.9465]])